# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [82]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import pprint from pprint
# Import API key
from api_keys import geoapify_key

SyntaxError: invalid syntax (3201063812.py, line 5)

In [48]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/City_Data.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,la'ie,21.6477,-157.9253,27.98,72,18,4.02,US,1712701012
1,1,warrnambool,-38.3833,142.4833,14.24,81,75,4.76,AU,1712701173
2,2,cabo san lucas,22.8909,-109.9124,27.56,43,0,2.06,MX,1712700969
3,3,polyarnyy,69.1989,33.4478,-3.84,92,100,1.63,RU,1712701185
4,4,port-aux-francais,-49.3500,70.2167,8.52,94,100,17.16,TF,1712701185


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [77]:
%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    size='Humidity',
    alpha = 0.3,
    geo = True,
    tiles = "OSM",
    color = "City",
    xlabel = "x",
    ylabel = "y",
    frame_width = 900,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [76]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[
    (city_data_df['Max Temp'].between(18, 28)) &
    (city_data_df['Humidity'].between(30, 90)) &
    (city_data_df['Wind Speed'] <= 5) 
]

# Drop any rows with null values
filtered_data = criteria_df.dropna()

# Display sample data
filtered_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,la'ie,21.6477,-157.9253,27.98,72,18,4.02,US,1712701012
2,2,cabo san lucas,22.8909,-109.9124,27.56,43,0,2.06,MX,1712700969
7,7,laguna,38.4210,-121.4238,25.62,39,0,2.06,US,1712701186
8,8,flying fish cove,-10.4217,105.6791,24.82,88,40,1.54,CX,1712700921
9,9,albany,42.6001,-73.9662,22.63,33,51,1.79,US,1712700918
...,...,...,...,...,...,...,...,...,...,...
526,526,ovalle,-30.5983,-71.2003,24.60,41,13,2.66,CL,1712701153
528,528,andalgala,-27.5819,-66.3165,19.40,59,65,1.39,AR,1712701279
537,537,muscatine,41.4667,-91.1335,18.10,34,40,3.60,US,1712701287
538,538,cambria,35.5641,-121.0808,26.34,62,0,4.30,US,1712701271


In [78]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = filtered_data[
    (filtered_data['Max Temp'].between(18, 28)) &
    (filtered_data['Humidity'].between(30, 90)) &
    (filtered_data['Wind Speed'] <= 5) 
]

# Drop any rows with null values

filtered_data = criteria_df.dropna()

filtered_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,la'ie,21.6477,-157.9253,27.98,72,18,4.02,US,1712701012
2,2,cabo san lucas,22.8909,-109.9124,27.56,43,0,2.06,MX,1712700969
7,7,laguna,38.4210,-121.4238,25.62,39,0,2.06,US,1712701186
8,8,flying fish cove,-10.4217,105.6791,24.82,88,40,1.54,CX,1712700921
9,9,albany,42.6001,-73.9662,22.63,33,51,1.79,US,1712700918
...,...,...,...,...,...,...,...,...,...,...
526,526,ovalle,-30.5983,-71.2003,24.60,41,13,2.66,CL,1712701153
528,528,andalgala,-27.5819,-66.3165,19.40,59,65,1.39,AR,1712701279
537,537,muscatine,41.4667,-91.1335,18.10,34,40,3.60,US,1712701287
538,538,cambria,35.5641,-121.0808,26.34,62,0,4.30,US,1712701271


### Step 3: Create a new DataFrame called `hotel_df`.

In [79]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({
    'City': filtered_data['City'], 
    'Country': filtered_data['Country'], 
    'Lat': filtered_data['Lat'], 
    'Lng': filtered_data['Lng'], 
    'Humidity': filtered_data['Humidity'], 
})

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,la'ie,US,21.6477,-157.9253,72,
2,cabo san lucas,MX,22.8909,-109.9124,43,
7,laguna,US,38.4210,-121.4238,39,
8,flying fish cove,CX,-10.4217,105.6791,88,
9,albany,US,42.6001,-73.9662,33,
...,...,...,...,...,...,...
526,ovalle,CL,-30.5983,-71.2003,41,
528,andalgala,AR,-27.5819,-66.3165,59,
537,muscatine,US,41.4667,-91.1335,34,
538,cambria,US,35.5641,-121.0808,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [83]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
params = {
    "catagories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "format":"json",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {longitude},{latitude},{radius}" # YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}"   # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params) # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json() # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
la'ie - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
flying fish cove - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
lamu - nearest hotel: No hotel found
ambodifotatra - nearest hotel: No hotel found
urubamba - nearest hotel: No hotel found
kerema - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
maitum - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
castillos - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
richards bay - nearest hotel: No hotel found
cidreira - nearest hotel: No hotel found
siwa oasis - nearest hotel: No hotel found
kourou - nearest hotel: No hotel found
san sebastian de la gomera - nearest hotel: No hotel found
mbengwi - nearest hotel: No hotel fou

,City,Country,Lat,Lng,Humidity,Hotel Name
0,la'ie,US,21.6477,-157.9253,72,No hotel found
2,cabo san lucas,MX,22.8909,-109.9124,43,No hotel found
7,laguna,US,38.4210,-121.4238,39,No hotel found
8,flying fish cove,CX,-10.4217,105.6791,88,No hotel found
9,albany,US,42.6001,-73.9662,33,No hotel found
...,...,...,...,...,...,...
526,ovalle,CL,-30.5983,-71.2003,41,No hotel found
528,andalgala,AR,-27.5819,-66.3165,59,No hotel found
537,muscatine,US,41.4667,-91.1335,34,No hotel found
538,cambria,US,35.5641,-121.0808,62,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [75]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    size='Humidity',
    geo = True,
    tiles = "OSM",
    color = "City",
    hover_cols = ["Hotel Name", "Country"],
    xlabel = "x",
    ylabel = "y",
    frame_width = 900,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)